<a href="https://colab.research.google.com/github/mahopman/IEMB-Net/blob/main/generate_evidence_integration_drug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
__author__ = 'Qiao Jin'
__editor__ = 'Mia Hopman'

In [2]:
local_path = '/content/drive/MyDrive/MS_DataScience/DS595/IEBM-Net_Data'
evidence_integration_path = f'{local_path}/evidence_integration'

In [ ]:
import json

def generate(picoids, split_path):
	output = []
	for picoid in picoids:
		if prompt_info[picoid]['label'] != 'invalid prompt':
			output.append({})
			output[-1]['picoid'] = picoid
			output[-1]['pmcid'] = prompt_info[picoid]['PMCID']
			output[-1]['i_text'] = prompt_info[picoid]['I']
			output[-1]['c_text'] = prompt_info[picoid]['C']
			output[-1]['o_text'] = prompt_info[picoid]['O']
			output[-1]['label'] = result2label[prompt_info[picoid]['label']]

			passage = ''
			if str(prompt_info[picoid]['PMCID']) in pmcid2content:
				content = pmcid2content[str(prompt_info[picoid]['PMCID'])]
				for secname, text in content:
					if secname[:len('ABSTRACT')] != 'ABSTRACT': continue
					if sec2label[secname2sec[secname]] == '1':
						passage += text

			output[-1]['passage'] = passage

	with open(split_path, 'w') as f:
		json.dump(output, f, indent=4)

In [21]:
import pandas as pd

prompts_df = pd.read_csv(f'{evidence_integration_path}/evidence_inference/prompts_merged.csv')
annotations_df = pd.read_csv(f'{evidence_integration_path}/evidence_inference/annotations_merged.csv')

annotations_df = annotations_df.loc[
    (annotations_df['Valid Label'] == True) &
    (annotations_df['Valid Reasoning'] == True)
]

annotations_df = annotations_df[['PromptID','Label']].drop_duplicates()

prompts_info_df = pd.merge(prompts_df, annotations_df, on='PromptID')

prompt_info = {}
for _, row in prompts_info_df.iterrows():
    picoid = row['PromptID']
    prompt_info[picoid] = {}
    prompt_info[picoid]['PMCID'] = row['PMCID']
    prompt_info[picoid]['I'] = row['Intervention']
    prompt_info[picoid]['C'] = row['Comparator']
    prompt_info[picoid]['O'] = row['Outcome']
    prompt_info[picoid]['label'] = row['Label']

with open(f'{evidence_integration_path}/evidence_inference/prompt_info.json', 'w') as f:
    json.dump(prompt_info, f, indent=4)

In [ ]:
result2label = {'significantly decreased': 0,\
				'no significant difference': 1,\
				'significantly increased': 2}

prompt_info = json.load(open(local_path + '/evidence_integration/materials/prompt_info.json'))
split2ids = json.load(open(local_path + '/evidence_integration/materials/split2ids.json'))
pmcid2picoid = json.load(open(local_path + '/evidence_integration/materials/pmcid2picoid.json'))
pmcid2content = json.load(open(local_path + '/evidence_integration/materials/pmc_contents.json'))
secname2sec = json.load(open(local_path + '/evidence_integration/materials/secname2sec.json'))
sec2label = json.load(open(local_path + '/evidence_integration/materials/sec2label.json'))

for split, ids in split2ids.items():
    picoids = []

    for pmcid in ids:
	    pmcid = str(pmcid)
        if pmcid in pmcid2picoid:
		    picoids += pmcid2picoid[pmcid]

    split_path = f"{local_path}/evidence_integration/{split}.json"
    generate(picoids, split_path)